### Author(s): evankoh


### Description
This notebook demonstrates:
1. How to retrieve historical prices for a stock
2. Compute moving averages
3. Plot results with line charts

### Import all neccessary libraries/functions

In [1]:
import os
os.chdir('../')
from stockscafe.StocksCafeApi import StocksCafeApi
import stockscafe.indicators.momentum.MovingAverage as MA
import pandas as pd
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

### Parameters (If unfamiliar with python, then only change the values in this cell)

In [18]:
exchange = 'SGX'
symbol = 'Z74'
name = 'Singtel'
shortMADays = 20
longMADays = 100
priceLookbackDays = 500 # Max is 1000

### Declaration & Get Recent Prices

In [19]:
init_notebook_mode(connected=True)
sc = StocksCafeApi()
df = sc.getPrices(exchange, symbol, priceLookbackDays)
df.tail()

,change,change_percent,close,currency,date,high,low,open,volume
495,-0.020,-0.51%,3.91,SGD,9 Mar 2017,3.92,3.89,3.92,"11,025,300"
496,0.030,0.77%,3.93,SGD,8 Mar 2017,3.95,3.89,3.90,"11,948,200"
497,0.010,0.26%,3.90,SGD,7 Mar 2017,3.93,3.88,3.89,"10,138,300"
498,-0.010,-0.26%,3.89,SGD,6 Mar 2017,3.92,3.87,3.90,"9,148,500"
499,-0.020,-0.51%,3.90,SGD,3 Mar 2017,3.90,3.85,3.90,"13,384,600"


### Compute Moving Averages

In [20]:
df = MA.compute(df, shortMADays) # Compute and add to Data Frame
df = MA.compute(df, longMADays) # Compute and add to Data Frame
df.tail() # Check data - Try 'df.head()'

,change,change_percent,close,currency,date,high,low,open,volume,sma20,sma100
495,0.010,0.33%,3.08,SGD,25 Feb 2019,3.09,3.07,3.08,"7,128,600",3.0335,3.0641
496,0.010,0.32%,3.09,SGD,26 Feb 2019,3.11,3.07,3.07,"21,677,900",3.0355,3.0627
497,-0.070,-2.27%,3.02,SGD,27 Feb 2019,3.10,3.02,3.08,"24,645,500",3.0350,3.0604
498,0,0%,3.02,SGD,28 Feb 2019,3.04,3.01,3.02,"24,434,800",3.0350,3.0582
499,-0.030,-0.99%,2.99,SGD,1 Mar 2019,3.01,2.97,2.99,"22,398,258",3.0340,3.0560


### Plot Graph

In [21]:
# Good resource to learn about graph basics - https://www.kaggle.com/kanncaa1/plotly-tutorial-for-beginners

traceClose = go.Scatter(
                    x = df.date,
                    y = df.close,
                    mode = "lines",
                    name = "Close",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'))

traceShortMA = go.Scatter(
                    x = df.date,
                    y = df[f'sma{shortMADays}'],
                    mode = "lines",
                    name = f'SMA-{shortMADays}',
                    marker = dict(size = 10, color = 'rgba(0, 0, 255, 0.3)'))

traceLongMA = go.Scatter(
                    x = df.date,
                    y = df[f'sma{longMADays}'],
                    mode = "lines",
                    name = f'SMA-{longMADays}',
                    marker = dict(color = 'rgba(255, 0, 0, 0.3)'))

data = [traceClose, traceShortMA, traceLongMA]

layout = dict(title = f'{name} Moving Average',
              xaxis = dict(title = '', ticklen = 0, zeroline = False, 
                           showticklabels = True, range = [0, len(df.index)], dtick = 100, tickwidth = 1),
              autosize = True, showlegend = False
             )

fig = dict(data = data, layout = layout)

iplot(fig)

### Generate and store graph in html

In [22]:
plot(fig, filename = f'output/{exchange}:{symbol}.html')

'file:///Users/ekoh/sgxcafe/Playground/output/SGX:Z74.html'